# Save Segmentation from RT Structures into labelmaps

## Example for DICOM series

### Load DICOM files into scene from directory

In [1]:
dicomDataDir = "/media/kamila/System/Users/Kama/Documents/Clara/Data/COOK/PROSTATE-20210427T133622Z-001/testy/CT" #dicomsource folder

def loadDICOMSeries(dicomDataDir):
    loadedNodeIDs = [] # this list will contain the list of all loaded node IDs
    from DICOMLib import DICOMUtils
    with DICOMUtils.TemporaryDICOMDatabase() as db:
        DICOMUtils.importDicom(dicomDataDir, db)
        patientUIDs = db.patients()
        for patientUID in patientUIDs:
            loadedNodeIDs.extend(DICOMUtils.loadPatientByUID(patientUID))

loadDICOMSeries(dicomDataDir)            
display()

Switching to temporary DICOM database: /tmp/Slicer-kamila/20210506_155316_TempDICOMDatabase
Loading with imageIOName: GDCM
Window/level found in DICOM tags (center=40.0, width=350.0) has been applied to volume 3: Unnamed Series
Window/level found in DICOM tags (center=40.0, width=350.0) has been applied to volume 3: Unnamed Series_1


### Load RT Struct file

In [3]:
contour_fn = '/media/kamila/System/Users/Kama/Documents/Clara/Data/COOK/PROSTATE-20210427T133622Z-001/testy/Structures/RS.1.2.246.352.71.4.787609392710.71605.20190812123433.dcm'

def loadRTStructFile(contour_fn):
    # Add filename to list for DICOM-RT module
    contour_vtkFileList = vtk.vtkStringArray()
    contour_vtkFileList.InsertNextValue(contour_fn)

    # Examine files
    contour_loadablesCollection = vtk.vtkCollection()
    slicer.modules.dicomrtimportexport.logic().ExamineForLoad(contour_vtkFileList, contour_loadablesCollection)
    # Set name for when loaded
    contour_loadablesCollection.GetItemAsObject(0).SetName('RTSTRUCT')

    # Load file
    contour_success = slicer.modules.dicomrtimportexport.logic().LoadDicomRT(contour_loadablesCollection.GetItemAsObject(0))

loadRTStructFile(contour_fn)

### Define: segmentationNode, segmentIds, labelmapVolumeNode, referenceVolumeNode

In [13]:
labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')  # create new


# segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode") # volume from which ...
# referenceVolumeNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLScalarVolumeNode")
referenceVolumeNode = slicer.mrmlScene.GetNodeByID('vtkMRMLScalarVolumeNode1')
segmentationNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLSegmentationNode")

segmentIds = vtk.vtkStringArray()
segmentNames = ["PTV"]
for segmentName in segmentNames:
    segmentId = segmentationNode.GetSegmentation().GetSegmentIdBySegmentName(segmentName)
    segmentIds.InsertNextValue(segmentId)




In [15]:
print(segmentationNode)

vtkMRMLSegmentationNode (0x634c630)
  ID: vtkMRMLSegmentationNode1
  ClassName: vtkMRMLSegmentationNode
  Name: RTSTRUCT
  Debug: false
  MTime: 548551
  Description: (none)
  SingletonTag: (none)
  HideFromEditors: false
  Selectable: true
  Selected: false
  UndoEnabled: false
  Node references:
    display [displayNodeRef]: vtkMRMLSegmentationDisplayNode1
    referenceImageGeometryRef: vtkMRMLScalarVolumeNode1
    storage [storageNodeRef]: (none)
    transform [transformNodeRef]: (none)
  Debug: Off
  Modified Time: 547456
  Reference Count: 1
  Registered Events: (none)
  Name = (none)
  RestoreSelectionState = 0
  TransformNodeID: (none)
  DisplayNodeIDs[0]: vtkMRMLSegmentationDisplayNode1
  Segmentation:    Debug: Off
    Modified Time: 1111094
    MasterRepresentationName:  Planar contour
    Number of segments:  18
    Segment: gr.krok+pech.nas
      Debug: Off
      Modified Time: 3167000
      Name: gr.krok+pech.nas
      Color: (1, 0, 0)
      NameAutoGenerated: true
      C

### Export Segments To Labelmap Node

In [16]:
exp_seg_sucees = slicer.vtkSlicerSegmentationsModuleLogic.ExportSegmentsToLabelmapNode(segmentationNode, segmentIds, labelmapVolumeNode, referenceVolumeNode)

In [17]:
print(exp_seg_sucees)

True


### Save node into file

In [18]:
save_lab_success = slicer.util.saveNode(labelmapVolumeNode,"/media/kamila/System/Users/Kama/Documents/Clara/Data/COOK/PROSTATE-20210427T133622Z-001/testy/Labelmaps/DCM/labels_nii/data.nii.gz")


In [19]:
print(save_lab_success)

True


## For whole dataset

### Load DICOM files into scene from directory

In [1]:
import os

folderData = '/media/kamila/System/Users/Kama/Documents/Clara/Data/COOK/PROSTATE-20210427T133622Z-001/PROSTATE'
listPatients = [ f.path for f in os.scandir(folderData) if f.is_dir() ]


def create_dir(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
        
# from: https://github.com/KeremTurgutlu/dicom-contour
def get_contour_file(path):
    """
    Get contour file from a given path by searching for ROIContourSequence 
    inside dicom data structure.
    More information on ROIContourSequence available here:
    http://dicom.nema.org/medical/dicom/2016c/output/chtml/part03/sect_C.8.8.6.html
    
    Inputs:
            path (str): path of the the directory that has DICOM files in it, e.g. folder of a single patient
    Return:
        contour_file (str): name of the file with the contour
    """
    # handle `/` missing
    if path[-1] != '/': path += '/'
    # get .dcm contour file
    fpaths = [path + f for f in os.listdir(path) if '.dcm' in f]
    n = 0
    contour_file = None
    for fpath in fpaths:
        f = pydicom.read_file(fpath)
        if 'ROIContourSequence' in dir(f):
            contour_file = fpath.split('/')[-1]
            n += 1
    if n > 1: warnings.warn("There are multiple contour files, returning the last one!")
    if contour_file is None: print("No contour file found in directory")
    return os.path.join(path, contour_file)


def find_ct_folder(path):
    return = path + '/CT'


def find_struct_folder(path):
    return = path + '/Structures'



for patient in listPatients:
    nifti_path = patient + '/nifti'
    nifti_imgs = nifti_path + '/img'
    nifti_lables = nifti_path + '/labels'
    create_dir(nifti_path)
    create_dir(nifti_imgs)
    create_dir(nifti_labels)
    CTFolder = find_ct_folder(patient)
    StrFolder = find_struct_folder(patient)
    StrFile = os.listdir(StrFolder)[0]
    loadDICOMSeries(CTFolder)
    loadRTStructFile(StrFile)
    labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')  # create new
    referenceVolumeNode = slicer.mrmlScene.GetNodeByID('vtkMRMLScalarVolumeNode1')
    segmentationNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLSegmentationNode")
    segmentIds = vtk.vtkStringArray()
    segmentNames = ["PTV"]
    for segmentName in segmentNames:
        segmentId = segmentationNode.GetSegmentation().GetSegmentIdBySegmentName(segmentName)
        segmentIds.InsertNextValue(segmentId)
    exp_seg_sucees = slicer.vtkSlicerSegmentationsModuleLogic.ExportSegmentsToLabelmapNode(segmentationNode, segmentIds, labelmapVolumeNode, referenceVolumeNode)
    save_lab_success = slicer.util.saveNode(labelmapVolumeNode, nifti_lables + "/data.nii.gz")


    

['/media/kamila/System/Users/Kama/Documents/Clara/Data/COOK/PROSTATE-20210427T133622Z-001/PROSTATE/STATIC_1', '/media/kamila/System/Users/Kama/Documents/Clara/Data/COOK/PROSTATE-20210427T133622Z-001/PROSTATE/STATIC_2']
